# Kuramoto-Sivashinsky Equation

$$
u_t = -u u_x - u_{xx} - \nu u_{xxxx}  \qquad (0 < x < L)
$$

$$
u_t = -\lambda u u_x - \lambda^2 u_{xx} - \nu \lambda^4 u_{xxxx}, \qquad (0 < x < 1)
$$

## Importing modules 

In [1]:
%matplotlib qt5

In [2]:
import time

In [3]:
import numpy as np
import equations as eq
from scipy.integrate import odeint
from scipy.fftpack import rfft, irfft

import h5py

In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Setting Parameters 

In [5]:
NW = 48
ks01 = eq.KS(NW)
args = ks01.getParamDefault()

In [6]:
N2 = ks01.N2
J = ks01.J
x = ks01.get_x()

## Setting initial data 

In [7]:
u0 = np.zeros(N2)
u0[1] = 0.5*J

## Computation 

In [8]:
trange0 = np.arange(0., 10.0, 2.0)
trange1 = np.arange(0., 490.0, 2.0)

maxdt = 1.0e-1

In [9]:
fname = 'ks.hdf5'

In [10]:
stime = time.time()
with h5py.File(fname, 'w') as fh:
    ks01.mkInitDataSet(u0, fh, maxdt)
    ks01.evolve(fh, trange0, tuple(args), method='RK23')
etime = time.time()
print(int(etime - stime))

36


In [11]:
stime = time.time()
with h5py.File(fname, 'r+') as fh:
    ks01.evolve(fh, trange1, tuple(args), method='RK23')
etime = time.time()
print(int(etime - stime))

4669


In [12]:
with h5py.File(fname, 'r') as fh:
    data = fh['u'][()]
    

In [13]:
up = ks01.mkPhysData(data)

C:\Users\rikota\Anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


## Visualization

In [14]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

ax.set_ylim((-3.0, 3.0))
ax.set_xlim((-0.1, 1.1))
ax.grid(True)
lines = plt.plot(x, [np.nan]*len(x));

In [15]:
def init():
    for l in lines:
        l.set_ydata([np.nan] * len(x))
    return lines

def animate(i):
    for l in lines:
        l.set_ydata(up[i,:])
    return lines

In [17]:
ani = animation.FuncAnimation(
    fig, animate, frames=up.shape[0],
    init_func=init, interval=100, blit=True,
    repeat=True)

plt.show()